In [ ]:
pip install openai==0.28

In [13]:
import openai
import pandas as pd
import os
import time
import re
# Set your OpenAI API key here
openai.api_key = 'fill in the API(delete here for public sharing of the code)'

# Function to evaluate the likelihood that an answer is by Rick
def evaluate_answer(question, answer):
    prompt = (
        f"Question: {question}\nAnswer: {answer}\n\n"
        "Evaluate the dialogue based on the following standards from 1 to 25 each, do not give the reason for each score:\n"
        "1. Vocabulary: Does the dialogue use language and terms that Rick would typically use?\n"
        "2. Creativity: Is the dialogue inventive and original, reflecting Rick's innovative and unconventional approach?\n"
        "3. Sentiment: Does the dialogue capture Rick's usual cynical, sarcastic, or nihilistic sentiment?\n"
        "4. Character Accuracy: Does the dialogue accurately reflect Rick's personality and behavior?\n\n"
        "Provide a total score out of 100."
    )
    
    while True:
        try:
            # Use the latest API method for creating completions
            response = openai.ChatCompletion.create(
                model="gpt-4o-2024-05-13",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=100
            )
            result = response['choices'][0]['message']['content'].strip()
            return result
        except openai.error.RateLimitError:
            print("Rate limit exceeded. Waiting for 60 seconds before retrying...")
            time.sleep(60)  # Wait for 60 seconds before retrying
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

# Change directory to where your CSV file is located
os.chdir(os.path.expanduser('~/Downloads/Inference'))

# Read the combined CSV file
file_path = 'combined_answers.csv'  # Replace with your CSV file path
df = pd.read_csv(file_path)

# Ensure the CSV has the correct columns
if 'Question' not in df.columns or not any(col.startswith('Answer_') for col in df.columns):
    raise ValueError("CSV file must contain 'Question' and 'Answer_' columns.")

# Add columns for scores
for col in df.columns:
    if col.startswith('Answer_'):
        df[f'{col}_Score'] = None
        
# Evaluate answers in batches of 10 to respect rate limits
batch_size = 10
for start in range(0, len(df), batch_size):
    end = min(start + batch_size, len(df))
    batch = df.iloc[start:end]

    # Loop through each row in the batch and evaluate each answer
    for index, row in batch.iterrows():
        question = row['Question']
        for col in df.columns:
            if col.startswith('Answer_'):
                answer = row[col]
                if pd.notna(answer):  # Check if the answer is not NaN
                    evaluation = evaluate_answer(question, answer)
                    score_column = f"{col}_Score"
                    df.at[index, score_column] = evaluation

# Save the results to a new CSV file
output_file_path = 'questions_answers_with_scores-4o-1.csv'
df.to_csv(output_file_path, index=False, encoding='utf-8')

print(f"Scores saved to {output_file_path}")

Scores saved to questions_answers_with_scores-4o-1.csv
